In [1]:
import calliope
import pandas as pd
import plotly.express as px

# We increase logging verbosity
calliope.set_log_verbosity("INFO", include_solver_output=False)

In [2]:
#model = calliope.examples.national_scale()

model = calliope.Model('model.yaml')

[2024-10-09 11:55:07] INFO     Model: initialising
[2024-10-09 11:55:07] INFO     Model: preprocessing stage 1 (model_run)
[2024-10-09 11:55:08] INFO     Model: preprocessing stage 2 (model_data)
[2024-10-09 11:55:08] INFO     Model: preprocessing complete


In [3]:
model.inputs

<xarray.Dataset> Size: 522kB
Dimensions:                       (costs: 1, techs: 6, nodes: 7, carriers: 3,
                                   timesteps: 744)
Coordinates:
  * costs                         (costs) object 8B 'monetary'
  * techs                         (techs) object 48B 'aluminum_demand' ... 'f...
  * carriers                      (carriers) object 24B 'aluminum' ... 'power'
  * nodes                         (nodes) object 56B 'Iceland' ... 'region2'
  * timesteps                     (timesteps) datetime64[ns] 6kB 2005-01-01 ....
Data variables: (12/35)
    objective_cost_weights        (costs) float64 8B 1.0
    bigM                          float64 8B 1e+06
    base_tech                     (techs) object 48B 'demand' ... 'supply'
    carrier_in                    (nodes, techs, carriers) float64 1kB nan .....
    color                         (techs) object 48B '#FF0000' ... '#00FF00'
    name                          (techs) object 48B 'Aluminum Demand' ... 'F...
    ...                            ...
    source_use_max                (timesteps, nodes, techs) float64 250kB nan...
    sink_use_equals               (timesteps, nodes, techs) float64 250kB nan...
    definition_matrix             (nodes, techs, carriers) bool 126B False .....
    distance                      (techs) float64 48B nan nan ... nan nan
    timestep_resolution           (timesteps) float64 6kB 1.0 1.0 ... 1.0 1.0
    timestep_weights              (timesteps) float64 6kB 1.0 1.0 ... 1.0 1.0
Attributes:
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev3
    applied_overrides:             
    scenario:                      None
    defaults:                      {'available_area': inf, 'bigM': 1000000000...
    allow_operate_mode:            1
    config:                        {'build': {'backend': 'pyomo', 'ensure_fea...
    applied_additional_math:       []
    math:                          {'constraints': {'flow_capacity_per_storag...
    name:                          2-node model

In [4]:
model.inputs.flow_cap_max.to_series().dropna()

techs
aluminum_supply                   inf
aluminum_transmission_line        inf
csp                           10000.0
fuel_supply                       inf
Name: flow_cap_max, dtype: float64

In [5]:
model.inputs.sink_use_equals.sum(
    "timesteps", min_count=1, skipna=True
).to_series().dropna()

nodes    techs       
region1  demand_power    2.444414e+07
region2  demand_power    1.786295e+06
Name: sink_use_equals, dtype: float64

In [6]:
model.build(force=True)
model.solve()

ModelError: Errors during model processing:
 * Must define node latitude and longitude for _all_ nodes or _no_ nodes.

In [ ]:
model.results